<a href="https://colab.research.google.com/github/snassimr/ApplicationsBuildWithLLMs/blob/main/deoptima_assistant_interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
%reload_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [3]:
SYS_PROJECT_DIR  = '/content/gdrive/MyDrive/Colab Notebooks/deoptima'
SYS_SRC_DIR  = '/content/gdrive/MyDrive/Colab Notebooks/deoptima/deoptima'
SYS_INPUT_DIR    = '/content/gdrive/MyDrive/Colab Notebooks/deoptima/input'
SYS_MODELING_DIR = '/content/gdrive/MyDrive/Colab Notebooks/deoptima/modeling'
SYS_TESTING_DIR = '/content/gdrive/MyDrive/Colab Notebooks/deoptima/testing'
SYS_RUNTIME_DIR = '/content/gdrive/MyDrive/Colab Notebooks/deoptima/runtime'
SYS_LLM_MODEL    = 'Mistral-7B-Instruct-v0.3'

In [4]:
import os
import pandas as pd
import shutil

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.options.mode.copy_on_write = True

# Installs

In [5]:
import os

requirements = """
vllm==0.5.4
gradio==4.41.0
accelerate==0.32.1
transformers==4.42.4
torch==2.3.1+cu121
"""
requirements = """
python-dotenv==1.0.1
gradio==4.43.0
pydantic-core==2.23.2
pydantic==2.9.0
fastapi==0.112.4
gradio-client==1.3.0
nemoguardrails==0.9.1.1
langchain-openai==0.1.21
"""
requirements = """
python-dotenv==1.0.1
gradio==4.44.1
nemoguardrails==0.9.1.1
langchain-openai==0.1.21
"""

requirements_path = os.path.join(SYS_PROJECT_DIR, 'requirements.txt')
with open(requirements_path, 'w') as f:
    f.write(requirements)

86

In [6]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [7]:
import os
requirements_path = os.path.join(SYS_PROJECT_DIR, 'requirements.txt')
!pip install -q -r '{requirements_path}'

In [8]:
import sys
import os

if SYS_PROJECT_DIR not in sys.path:
    sys.path.append(SYS_PROJECT_DIR)

In [9]:
from deoptima.utils import generate_session_id
session_id = generate_session_id()
# import os
# os.environ["SESSION_ID"] = session_id

In [10]:
from deoptima.utils import set_logger
deoptima_logger = set_logger(session_id, log_mode = 'FS')

In [11]:
# import logging
# deoptima_logger = logging.getLogger("deoptima")

In [12]:
# import os
# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
# openai_api_key = userdata.get("OPENAI_API_KEY")

In [13]:
from dotenv import load_dotenv, find_dotenv
# Get the current environment, default to 'dev'
ENV = os.getenv('ENV', 'dev')

# Load the appropriate .env file based on the ENV variable
if ENV == 'prod':
  _ = load_dotenv(os.path.join(SYS_PROJECT_DIR,'./.env.prod'))
else:
  _ = load_dotenv(os.path.join(SYS_PROJECT_DIR,'./.env.dev'))

# Assistant Chain

In [14]:
from deoptima.assistant import create_assistant_chain

# assistant_chain, chain_config = get_assistant_chain(session_id)

create_assistant_chain(session_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

# Run Gradio Assistant

## Imports

In [15]:
import sys
import os
import gradio as gr
import random
import numpy as np
import pandas as pd

from deoptima.assistant import get_assistant_response

##Sample Data

In [16]:
sample_process_state = {
    'user_goal': 'selecting the best laptop',
    'options_list': ['Laptop 1', 'Laptop 2', 'Laptop 3'],
    'criteria_list': ['Price', 'Performance', 'Battery Life'],
    'criteria_comparison_list':
        [{'criterion_1': 'Price', 'criterion_2': 'Performance', 'criteria_rating': 'Strongly More Important'},
        {'criterion_1': 'Price', 'criterion_2': 'Battery Life', 'criteria_rating': 'Moderately More Important'},
        {'criterion_1': 'Performance', 'criterion_2': 'Battery Life', 'criteria_rating': 'Very Strongly More Important'}],
    'preferences_dictionary' : {
        'Price':
            [{'option_1': 'Laptop 1', 'option_2': 'Laptop 2', 'preference_rating': 'Moderately Less Preferred'},
             {'option_1': 'Laptop 1', 'option_2': 'Laptop 3', 'preference_rating': 'Strongly More Preferred'},
             {'option_1': 'Laptop 2', 'option_2': 'Laptop 3', 'preference_rating': 'Moderately More Preferred'}],
        'Performance' :
            [{'option_1': 'Laptop 1', 'option_2': 'Laptop 2', 'preference_rating': 'Moderately Less Preferred'},
             {'option_1': 'Laptop 1', 'option_2': 'Laptop 3', 'preference_rating': 'Strongly Less Preferred'},
             {'option_1': 'Laptop 2', 'option_2': 'Laptop 3', 'preference_rating': 'Moderately More Preferred'}],
        'Battery Life' :
            [{'option_1': 'Laptop 1', 'option_2': 'Laptop 2', 'preference_rating': 'Moderately Less Preferred'},
             {'option_1': 'Laptop 1', 'option_2': 'Laptop 3', 'preference_rating': 'Very Strongly Less Preferred'},
             {'option_1': 'Laptop 2', 'option_2': 'Laptop 3', 'preference_rating': 'Extremely More Preferred'}],
        }
    }

## Functions

In [17]:
def process_state_to_dfs(process_state : dict):

  import pandas as pd

  ### sync with base process_state
  if (process_state['user_goal'] is None or process_state['user_goal'] == ''):
      process_state['user_goal'] = 'Missing'
  if (process_state['options_list'] is None or len(process_state['options_list']) == 0):
      process_state['options_list'] = ['Missing']
  if (process_state['criteria_list'] is None or len(process_state['criteria_list']) == 0):
      process_state['criteria_list'] = ['Missing']
  if ('preferences_dictionary' not in process_state.keys()):
      process_state['preferences_dictionary'] = {
      'Missing criteria list':
          [],
      }
  if (len(process_state['preferences_dictionary'].keys()) == 0):
      process_state['preferences_dictionary'] = {
          'Missing criteria list':
              [],
          }

  basic_data_dict = {key: process_state[key] for key in ['user_goal', 'options_list', 'criteria_list']}
  basic_data_df = pd.DataFrame(list(basic_data_dict.items()), columns=['Setting', 'Value'])
  basic_data_df['Setting'] = ['Goal','Options',"Criteria"]

  criteria_df = pd.DataFrame(process_state['criteria_comparison_list'], columns=['criterion_1', 'criteria_rating', 'criterion_2'])

  preferences_dict = process_state['preferences_dictionary']

  for k,v in preferences_dict.items():
      criterion_preferences_df = pd.DataFrame(v, columns=['option_1', 'preference_rating', 'option_2'])
      preferences_dict[k] = criterion_preferences_df

  return basic_data_df, criteria_df, preferences_dict

In [18]:
def initialize_dfs():

  import copy
  import pandas as pd

  global curr_assistant_message, curr_user_examples
  global curr_process_state, curr_basic_data_df, curr_criteria_df, curr_preferences_dict

  curr_assistant_message = None

  curr_user_examples = ["I'm ready to start the decision-making process ⛏"]

  curr_process_state = {
    'user_goal': 'Missing',
    'options_list': ['Missing'],
    'criteria_list': ['Missing'],
    'criteria_comparison_list':
        [],
    'preferences_dictionary' : {
        'Missing criteria list':
            [],
        }
    }

  curr_basic_data_df, curr_criteria_df, curr_preferences_dict = process_state_to_dfs(curr_process_state)

initialize_dfs()

In [19]:
import copy
from deoptima.assistant import print_history

def get_chat_sample_assistant_response(message, history):

  import time

  global curr_assistant_message, curr_user_examples
  global curr_process_state, curr_basic_data_df, curr_criteria_df, curr_preferences_dict

  assistant_message = random.choice(["How are you?", "I love you", "I'm very hungry"])

  user_examples_list = [
      ["User example 1"], ["User example 2"], ["User example 3"], ["User example 4"], ["User example 5"]
      ]
  user_examples = random.sample(user_examples_list, 3)

  process_state = copy.deepcopy(sample_process_state)

  curr_assistant_message = assistant_message
  curr_user_examples = user_examples
  curr_process_state = copy.deepcopy(process_state)

  print_history(message, curr_assistant_message, history)

  curr_basic_data_df, curr_criteria_df, curr_preferences_dict = process_state_to_dfs(curr_process_state)

  time.sleep(0.5)


def get_chat_assistant_response(message, history):

  from deoptima.assistant import set_predefined_assistant_message

  global curr_assistant_message, curr_user_examples
  global curr_process_state, curr_basic_data_df, curr_criteria_df, curr_preferences_dict

  assistant_message, user_examples, process_state = get_assistant_response(message)

  assistant_message = set_predefined_assistant_message(assistant_message)
  curr_assistant_message = assistant_message

  print(f"****************************{type(user_examples)}")

  curr_user_examples = [[item] for item in user_examples]

  # if isinstance(user_examples, list):
  #     curr_user_examples = [[item] for item in user_examples]
  # elif isinstance(user_examples, str):
  #     curr_user_examples = [[user_examples]]
  # else:
  #     curr_user_examples = []

  if(process_state):
    curr_process_state = copy.deepcopy(process_state)

  # print_history(message, curr_assistant_message, history)

  curr_basic_data_df, curr_criteria_df, curr_preferences_dict = process_state_to_dfs(curr_process_state)


get_chat_assistant_response = get_chat_assistant_response

## Launch interface

In [20]:
css = """
/* Hide sort buttons at gr.DataFrame */
.sort-button {
    display: none !important;
}
footer{display:none !important}
"""
with gr.Blocks(
    theme="soft",
    css = css,
    fill_height=True,fill_width=True,
    ) as interface:

    chatbot = gr.Chatbot(placeholder="Are you ready to start decision-making process ?", label = "Deoptima", scale=1)

    message_box = None
    submit_button = None
    curr_user_message = None

    # def handle_submit(message, chat_history):
    #     get_chat_assistant_response(message, chat_history)
    #     chat_assistant_message = f"✨\n {curr_assistant_message}"
    #     user_message = f"👥\n{message}"
    #     chat_history.append((user_message, chat_assistant_message))
    #     return "", chat_history

    def handle_user_message(user_message, chat_history):
        global curr_user_message
        curr_user_message = user_message
        chat_user_message = f"👥\n{curr_user_message}"
        chat_history.append([chat_user_message, None])
        return "", chat_history

    def handle_assistant_message(chat_history):
        global curr_user_message
        get_chat_assistant_response(curr_user_message, chat_history)
        chat_assistant_message = f"✨\n{curr_assistant_message}"

        if chat_history and chat_history[-1][1] is None:
            chat_history[-1][1] = chat_assistant_message
        else:
            chat_history.append([None, chat_assistant_message])

        return "", chat_history

    with gr.Row():
      message_box = gr.Textbox(placeholder="Send message to Deoptima", container=False, scale=7)
      submit_button = gr.Button("▶️ Submit", scale=2, interactive=False)
      # submit_event = submit_button.click(handle_submit, inputs=[message_box, chatbot], outputs=[message_box, chatbot])
      submit_event = submit_button.click(handle_user_message, inputs=[message_box, chatbot], outputs=[message_box, chatbot])
      submit_event_then = submit_event.then(
            handle_assistant_message,
            inputs=[chatbot], outputs=[message_box, chatbot]
        )
      # submit_event = submit_button.click(handle_user_message, inputs=[message_box, chatbot], outputs=[message_box, chatbot]).then(
      #       handle_assistant_message,
      #       inputs=[chatbot], outputs=[message_box, chatbot]
      #   )

      def enable_submit_button(message):
          return gr.update(interactive=bool(message.strip()))
      message_box_event = message_box.change(enable_submit_button, inputs=message_box, outputs=submit_button)


    examples = gr.Examples(examples = curr_user_examples, inputs = message_box, label="Sample answers")

    def handle_chatbot_change(chat_history, examples, *args):
      return_tuple = ()
      return_tuple = return_tuple + (chat_history,)
      return_tuple = return_tuple + (gr.Dataset(samples=curr_user_examples),)
      # Update Dataframes
      return_tuple = return_tuple + (gr.DataFrame(value = curr_basic_data_df),)
      return_tuple = return_tuple + (gr.DataFrame(value = curr_criteria_df),)
      return return_tuple + (curr_preferences_dict,)

    with gr.Group():

      preferences_state = gr.State(curr_preferences_dict)

      with gr.Tab("Basic Data"):
        df_gr_basic = gr.DataFrame(
            value = curr_basic_data_df,
            label=None,
            headers= None,
            interactive=False,
            wrap=False
            )
      with gr.Tab("Criteria"):
        df_gr_criteria = gr.DataFrame(
            value = curr_criteria_df,
            label=None,
            headers= None,
            interactive=False,
            wrap=False
            )
      with gr.Tab("Preferences"):
        @gr.render(inputs=preferences_state)
        def render_preferences(state):
          for k,v in state.items():
              gr.DataFrame(
                  value=v,
                  label=k,
                  headers=None,
                  interactive=False,
                  scale = 1,
                  wrap=False,
                  column_widths=[30,40,30],
                  )

    chatbot_change_event = chatbot.change(handle_chatbot_change,
                                          inputs=[chatbot, examples.dataset] + \
                                                [df_gr_basic] + [df_gr_criteria] + [preferences_state],
                                          outputs=[chatbot, examples.dataset] + \
                                                  [df_gr_basic] + [df_gr_criteria] + [preferences_state])
interface.launch(debug=True)




Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://8d17d6ddd07933dc18.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


deoptima - INFO     : 2024-11-13 16:29:04,678 : **** get_assistant_response started.
deoptima - INFO     : 2024-11-13 16:29:04,680 : input *** run_id: 0, user_message: I'm ready to start the decision-making process ⛏
nemoguardrails.rails.llm.llmrails - INFO     : 2024-11-13 16:29:06,754 : --- :: Total processing took 1.26 seconds. LLM Stats: 1 total calls, 1.24 total time, 193 total tokens, 192 total prompt tokens, 1 total completion tokens, [1.24] as latencies
deoptima - INFO     : 2024-11-13 16:29:06,758 : generate_model_guardials_response started ...
deoptima - INFO     : 2024-11-13 16:29:09,309 : generate_model_guardrials_response finished: 
--- model_gr_response ---
model_gr_class:
	start_process
assistant_message:
	<guidelines>
user_examples:
	- I am ready to start with definitions for Deoptima
deoptima - INFO     : 2024-11-13 16:29:09,443 : output_dict : {'input_gr_check_pass': True, 'model_gr_class': 'start_process', 'model_gr_ai_message': '<guidelines>', 'user_examples': ['I a

****************************<class 'list'>


deoptima - INFO     : 2024-11-13 16:31:27,739 : **** get_assistant_response started.
deoptima - INFO     : 2024-11-13 16:31:27,740 : input *** run_id: 1, user_message: I am ready to start with definitions for Deoptima
nemoguardrails.rails.llm.llmrails - INFO     : 2024-11-13 16:31:28,199 : --- :: Total processing took 0.45 seconds. LLM Stats: 1 total calls, 0.42 total time, 193 total tokens, 192 total prompt tokens, 1 total completion tokens, [0.42] as latencies
deoptima - INFO     : 2024-11-13 16:31:28,203 : generate_model_guardials_response started ...
deoptima - INFO     : 2024-11-13 16:31:32,816 : generate_model_guardrials_response finished: 
--- model_gr_response ---
model_gr_class:
	in_process
assistant_message:
	process
user_examples:
deoptima - INFO     : 2024-11-13 16:31:34,994 : get_process_state - get_response_data finished: 
--- process_state ---
user_goal:
	Missing
options_list:
criteria_list:
criteria_comparison_list:
preferences_dictionary:
deoptima - INFO     : 2024-11-

****************************<class 'list'>


...

****************************<class 'list'>


deoptima - INFO     : 2024-11-13 16:42:48,661 : **** get_assistant_response started.
deoptima - INFO     : 2024-11-13 16:42:48,662 : input *** run_id: 15, user_message: 'Interior Space': 'Sedan' is Moderately More Preferred than 'Hatchback'
nemoguardrails.rails.llm.llmrails - INFO     : 2024-11-13 16:42:50,891 : --- :: Total processing took 2.22 seconds. LLM Stats: 1 total calls, 2.14 total time, 200 total tokens, 199 total prompt tokens, 1 total completion tokens, [2.14] as latencies
deoptima - INFO     : 2024-11-13 16:42:50,894 : generate_model_guardials_response started ...
deoptima - INFO     : 2024-11-13 16:42:56,067 : generate_model_guardrials_response finished: 
--- model_gr_response ---
model_gr_class:
	in_process
assistant_message:
	process
user_examples:
deoptima - INFO     : 2024-11-13 16:43:03,647 : get_process_state - get_response_data finished: 
--- process_state ---
user_goal:
	Finding the most suitable car for my needs
options_list:
	- Sedan
	- SUV
	- Hatchback
criteria

****************************<class 'list'>


deoptima - INFO     : 2024-11-13 16:43:45,792 : **** get_assistant_response started.
deoptima - INFO     : 2024-11-13 16:43:45,793 : input *** run_id: 16, user_message: 'Interior Space': 'SUV' is Moderately More Preferred than 'Hatchback'
nemoguardrails.rails.llm.llmrails - INFO     : 2024-11-13 16:43:46,291 : --- :: Total processing took 0.49 seconds. LLM Stats: 1 total calls, 0.4 total time, 199 total tokens, 198 total prompt tokens, 1 total completion tokens, [0.4] as latencies
deoptima - INFO     : 2024-11-13 16:43:46,295 : generate_model_guardials_response started ...
deoptima - INFO     : 2024-11-13 16:43:49,348 : generate_model_guardrials_response finished: 
--- model_gr_response ---
model_gr_class:
	in_process
assistant_message:
	process
user_examples:
deoptima - INFO     : 2024-11-13 16:43:56,850 : get_process_state - get_response_data finished: 
--- process_state ---
user_goal:
	Finding the most suitable car for my needs
options_list:
	- Sedan
	- SUV
	- Hatchback
criteria_lis

****************************<class 'list'>
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://8d17d6ddd07933dc18.gradio.live


In [21]:
from deoptima.assistant import get_assistant_chain
from deoptima.assistant import print_langchain_history
assistant_chain, chain_config = get_assistant_chain()
print_langchain_history(assistant_chain.get_session_history(session_id))


User: I'm ready to start the decision-making process ⛏
Assistant (JSON):
{
    "input_gr_check_pass": true,
    "model_gr_class": "start_process",
    "model_gr_ai_message": "<guidelines>",
    "user_examples": [
        "I am ready to start with definitions for Deoptima"
    ]
}
User: I am ready to start with definitions for Deoptima
Assistant (JSON):
{
    "user_message": "I am ready to start with definitions for Deoptima",
    "input_gr_check_pass": true,
    "model_gr_class": "in_process",
    "model_gr_ai_message": "process",
    "model_ai_message": "Great! Let's start by defining your goal. What is the goal you are trying to achieve in an optimal way?",
    "user_examples": [
        "Selecting the best laptop",
        "Choosing the ideal vacation destination",
        "Finding the most suitable car for my needs"
    ],
    "all_validity_checks_passed": false,
    "process_state": {
        "user_goal": "Missing",
        "options_list": [],
        "criteria_list": [],
       

# Sleep

In [22]:
# import time
# time.sleep(30000)